In [96]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

# Keras neural network model
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from keras import optimizers, regularizers, initializers
from keras.models import Sequential, save_model, load_model
from keras.layers import Dense, Dropout, BatchNormalization

# Scikit-sklearn preprocessing modules
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Matplotlib and plotly modules
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

In [64]:
df_deviatoric = pd.read_csv("data/deviatoric.csv")
df_deviatoric_nsw = pd.read_csv("data/deviatoric_nsw.csv")
df_total = pd.read_csv("data/total.csv")
df_total_nsw = pd.read_csv("data/total_nsw.csv")

In [65]:
df_total.columns

Index(['velocity:0', 'velocity:1', 'force:0', 'force:1',
       'total displacement:0', 'total displacement:1', 'temperature',
       'temp_power', 'temp_pressure', 'temp_density', 'plastic strain',
       'plastic strain-rate', 'strain-rate II log10', 'strain-rate XX',
       'elastic_strain XX', 'strain-rate ZZ', 'elastic_strain ZZ',
       'strain-rate XZ', 'elastic_strain XZ', 'strain I', 'strain II',
       'strain XX', 'strain ZZ', 'strain XZ', 'stress I', 'stress II',
       'stress XX', 'stress ZZ', 'stress XZ', 'density', 'thermal_stress',
       'energy_total_vol_dev', 'energy_volumetric', 'energy_deviatoric',
       'energy_thermal', 'energy_elastic', 'coordinate:0', 'coordinate:1'],
      dtype='object')

In [66]:
threshold = 0.15
df_deviatoric['plastic strain'] = np.where(df_deviatoric['plastic strain'] > threshold, 1,0)
df_deviatoric_nsw['plastic strain'] = np.where(df_deviatoric_nsw['plastic strain'] > threshold, 1,0)
df_total['plastic strain'] = np.where(df_total['plastic strain'] > threshold, 1,0)
df_total_nsw['plastic strain'] = np.where(df_total_nsw['plastic strain'] > threshold, 1,0)

In [67]:
def drop_and_split_coloumns(df):
    
    dropped_cols = ['plastic strain', 'plastic strain-rate', 'strain-rate II log10', 'strain-rate XX',
           'elastic_strain XX', 'strain-rate ZZ', 'elastic_strain ZZ',
           'strain-rate XZ', 'elastic_strain XZ', 'strain I', 'strain II',
           'strain XX', 'strain ZZ', 'strain XZ', 'coordinate:0', 'coordinate:1', 'force:0', 'force:1', 
           'total displacement:0', 'total displacement:1', 'stress I', 'stress II', 'energy_volumetric', 
           'energy_deviatoric','energy_thermal', 'energy_elastic', 'thermal_stress',
           'energy_total_vol_dev', 'temp_power', 'temp_pressure', 'temp_density']

    Y = df['plastic strain']
    df = df.drop(dropped_cols, axis=1)
    X = df
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)
    
    return X_train, X_test, y_train, y_test, df

In [68]:
X_train_dev, X_test_dev, Y_train_dev, Y_test_dev, df_deviatoric = drop_and_split_coloumns(df_deviatoric)
X_train_dev_nsw, X_test_dev_nsw, Y_train_dev_nsw, Y_test_dev_nsw, df_deviatoric_nsw = drop_and_split_coloumns(df_deviatoric_nsw)
X_train_tot, X_test_tot, Y_train_tot, Y_test_tot, df_total = drop_and_split_coloumns(df_total)
X_train_tot_nsw, X_test_tot_nsw, Y_train_tot_nsw, Y_test_tot_nsw, df_total_nsw = drop_and_split_coloumns(df_total_nsw)

In [69]:
xtrains = [X_train_dev, X_train_dev_nsw, X_train_tot, X_train_tot_nsw]
xtests = [X_test_dev, X_test_dev_nsw, X_test_tot, X_test_tot_nsw]
ytrains = [Y_train_dev, Y_train_dev_nsw, Y_train_tot, Y_train_tot_nsw]
ytests = [Y_test_dev, Y_test_dev_nsw, Y_test_tot, Y_test_tot_nsw]
models_name = ['Deviatoric', 'Deviatoric_nsw', 'Total', 'Total_nsw']

In [70]:
df_total.columns

Index(['velocity:0', 'velocity:1', 'temperature', 'stress XX', 'stress ZZ',
       'stress XZ', 'density'],
      dtype='object')

In [71]:
def scale_feature(features):    
    mms = StandardScaler()
    scaled_features = mms.fit_transform(features)
    return scaled_features, mms

In [72]:
for i in range(len(models_name)):
    xtrains[i] = scale_feature(xtrains[i])
    xtests[i] = scale_feature(xtests[i])

## Neural network model

In [105]:
class build_neural_network:
    
    def __init__(self, model_name, xtrain, ytrain, xtest, ytest, training_epochs):
        self.model_name = model_name
        self.xtrain = xtrain
        self.ytrain = to_categorical(ytrain)
        self.xtest = xtest
        self.ytest = to_categorical(ytest)
        self.input_shape = np.shape(xtrain)[1]
        self.output_shape = np.shape(self.ytrain)[1]
        self.model = None
        self.ypred = None
        self.training_epochs = training_epochs
        self.weight_path = "./weights/"
            
    def get_saved_model(self):
        loaded_model = load_model('./logs/')
        loaded_model.load_weights('./logs/random_seed_1_999.hdf5')
        return loaded_model

    def create_model(self):
        self.model = Sequential()
        self.model.add(Dense(self.input_shape, 
                             kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=0.05, seed = 2018), 
                             activation = 'relu', name='layer_1',
                             input_shape = (self.input_shape,)))
        self.model.add(Dense(self.output_shape, activation="softmax", name='Output_layer'))
        self.model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
    
    def predict(self):
        self.create_model()
        filepath = self.weight_path + '_' + str(self.model_name) + '.hdf5'
        checkpoint = ModelCheckpoint(filepath, verbose=0, period = self.training_epochs)
        self.model.fit(self.xtrain, self.ytrain, batch_size = 10, epochs = self.training_epochs, 
        callbacks = [checkpoint],
        validation_split = 0.3)
    
    def classification_report(self):
        self.ypred = self.model.predict(self.xtest)
        Y_test_class = np.argmax(self.ytest, axis = 1)
        Y_pred_class = np.argmax(self.ypred, axis = 1)
        print('Accuracy: {}'.format(accuracy_score(Y_test_class, Y_pred_class)))
        print('Classification report: \n {} \n'.format(classification_report(Y_test_class, Y_pred_class)))
        print('Confusion matrix: {}'.format(confusion_matrix(Y_test_class, Y_pred_class)))
        

In [106]:
deviatoric = build_neural_network(models_name[0], xtrains[0][0], ytrains[0], xtests[0][0], ytests[0], 1)

In [107]:
deviatoric.predict()

Train on 1303478 samples, validate on 558634 samples
Epoch 1/1
1303454/1303478 [============================>.] - ETA: 0s - loss: 0.0639 - acc: 0.9773

OSError: Unable to create file (Unable to open file: name = './weights/_deviatoric.hdf5', errno = 2, error message = 'no such file or directory', flags = 13, o_flags = 602)

In [108]:
deviatoric.classification_report()

0.978823328923
             precision    recall  f1-score   support

          0       0.98      0.97      0.98    351678
          1       0.98      0.99      0.98    446370

avg / total       0.98      0.98      0.98    798048



array([[341321,  10357],
       [  6543, 439827]])